In [2]:
import os
import tensorflow as tf
import numpy as np
#import pandas as pd
#import matplotlib.pyplot as plt
from tensorflow import keras

In [3]:
# Will download and load `imagenet` pretrained weights.
# Model weight is loaded with `by_name=True, skip_mismatch=True`.
# include_preprocessing set True will add pre-processing Rescale + Normalization after Input. 
# Means using input value in range [0, 255]. Default value False means in range [-1, 1].

from keras_cv_attention_models import efficientnet
model3 = efficientnet.EfficientNetV2B0(pretrained="imagenet", include_preprocessing=True) # value in range [0, 255]
model2 = efficientnet.EfficientNetV2B0(pretrained="imagenet") # values in range [-1, 1]
#model = keras.models.load_model("/home/local/lambda/rishabhs/ML/keras_cv_attention_models/checkpoints_bkup_finetunedcifar10")
pretrained = os.path.expanduser('/home/local/lambda/rishabhs/ML/keras_cv_attention_models/checkpoints_bkup_finetunedcifar10/effv2b0_cifar10_224_progressive_epoch_35_val_acc_0.9528.h5')
model = efficientnet.EfficientNetV2B0(pretrained=pretrained, input_shape=(224, 224, 3), num_classes=10, classifier_activation='softmax', dropout=0.1)
model.summary()


/home/local/lambda/rishabhs/anaconda3/envs/cloned_tf2.8-cudnn8.1-cudadev-11.2-py3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-04-06 02:16:00.453597: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-06 02:16:02.404324: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 47227 MB memory:  -> device: 0, name: Quadro RTX 8000, pci bus id: 0000:19:00.0, compute capability: 7.5
2022-04-06 02:16:02.406406: I tensorflow/core/common_runtime/gpu/gpu_device.cc

>>>> Load pretrained from: /home/unga/rishabhs/.keras/models/efficientnetv2/efficientnetv2-b0-imagenet.h5
>>>> Load pretrained from: /home/unga/rishabhs/.keras/models/efficientnetv2/efficientnetv2-b0-imagenet.h5
>>>> Load pretrained from: /home/local/lambda/rishabhs/ML/keras_cv_attention_models/checkpoints_bkup_finetunedcifar10/effv2b0_cifar10_224_progressive_epoch_35_val_acc_0.9528.h5
Model: "efficientnet_v2-b0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv (Conv2D)             (None, 112, 112, 32  864         ['input_3[0][0]']      

In [8]:
""" Run predict """
import tensorflow as tf
from tensorflow import keras
from skimage.data import chelsea

# Run prediction
# Needs preprocessing
imm = tf.image.resize(chelsea(), model.input_shape[1:3]) # Chelsea the cat
pred = model(tf.expand_dims(imm / 128. - 1., 0)).numpy()
#pred = model(tf.expand_dims(imm, 0)).numpy()
#print('model', keras.applications.efficientnet_v2.decode_predictions(pred)[0])

img = chelsea() # Chelsea the cat
imm = keras.applications.imagenet_utils.preprocess_input(img, mode='torch') # preprocesses the data to [-1, 1]
pred2 = model2(tf.expand_dims(tf.image.resize(imm, model2.input_shape[1:3]), 0)).numpy()
print('model2', keras.applications.imagenet_utils.decode_predictions(pred2)[0])

# imm = tf.image.resize(chelsea(), model3.input_shape[1:3]) # Chelsea the cat
# pred = model(tf.expand_dims(imm, 0)).numpy()  # value in range [0, 255]
img = chelsea() # Chelsea the cat
pred3 = model3(tf.expand_dims(tf.image.resize(img, model3.input_shape[1:3]), 0)).numpy()
print('model3', keras.applications.imagenet_utils.decode_predictions(pred3)[0])

#pred = tf.nn.softmax(pred).numpy()
#pred = model(tf.expand_dims(imm / 128. - 1., 0)).numpy()


model2 [('n02124075', 'Egyptian_cat', 0.6614582), ('n02123159', 'tiger_cat', 0.13258176), ('n02123045', 'tabby', 0.10972807), ('n02127052', 'lynx', 0.0113952765), ('n04209133', 'shower_cap', 0.001039359)]
model3 [('n02124075', 'Egyptian_cat', 0.6614583), ('n02123159', 'tiger_cat', 0.13258165), ('n02123045', 'tabby', 0.10972804), ('n02127052', 'lynx', 0.011395274), ('n04209133', 'shower_cap', 0.0010393584)]


In [17]:
%%bash

CUDA_VISIBLE_DEVICES='1' python ./eval_script.py -m ./checkpoints/effv2b0_cifar10_224_progressive_epoch_35_val_acc_0.9528.h5 --central_crop 0.875 -d cifar10 


2022-04-06 13:48:47.599598: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-06 13:48:48.050885: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46904 MB memory:  -> device: 0, name: Quadro RTX 8000, pci bus id: 0000:1a:00.0, compute capability: 7.5


>>>> Using input_shape (224, 224) for Keras model.
>>>> rescale_mode: torch
>i>>> Dataset args: {'data_name': 'cifar10', 'batch_size': 64, 'buffer_size': 1000, 'info_only': False, 'rescale_mode': 'torch', 'random_crop_min': 1.0, 'random_erasing_prob': 0.0, 'magnitude': 0, 'num_layers': 2, 'augment_kwargs': {}, 'cutmix_alpha': 0, 'eval_central_crop': 0.875, 'input_shape': (224, 224), 'mixup_alpha': 0, 'resize_antialias': True, 'resize_method': 'bicubic'}


Evaluating: 100%|██████████| 157/157 [00:17<00:00,  9.05it/s]


>>>> Accuracy top1: 0.9524 top5: 0.9987


### Conv2d to Split Conv2d
* Once the model is converted the new model outputs for a set of inputs is compared with the original model outputs.

### Model converted to a TFLite model (No quantization done)
* Again the converted the model outputs for a set of inputs is compared with the Split Conv2d model.

### Is split conv2d really necessary?

In [9]:
from keras_cv_attention_models import regnet, model_surgery
from keras_cv_attention_models.imagenet import eval_func

mm = model_surgery.convert_groups_conv2d_2_split_conv2d(model)  # converts all `Conv2D` using `groups` to `SplitConv2D`
#mm = model
test_inputs = np.random.uniform(size=[1, *mm.input_shape[1:]])
print(np.allclose(mm(test_inputs), model(test_inputs)))
# True

converter = tf.lite.TFLiteConverter.from_keras_model(mm)
open(mm.name + "_no_quant.tflite", "wb").write(converter.convert())
print(np.allclose(mm(test_inputs), eval_func.TFLiteModelInterf(mm.name + '_no_quant.tflite')(test_inputs), atol=1e-7))
# True

True


2022-04-06 02:34:48.636972: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmphmbkwcpf/assets


2022-04-06 02:35:16.481160: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-04-06 02:35:16.481187: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-04-06 02:35:16.482185: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmphmbkwcpf
2022-04-06 02:35:16.525747: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-04-06 02:35:16.525772: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmphmbkwcpf
2022-04-06 02:35:16.727623: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-04-06 02:35:17.442585: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmphmbkwcpf
2022-04-06 02:35:17.789617: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 1307436

True


### Accuracy top1: 0.9523 top5: 0.9987
* CUDA_VISIBLE_DEVICES='1,2,3,4' python ./eval_script.py -m ./efficientnet_v2-b0_no_quant.tflite --central_crop 0.875 -d cifar10 
* 23 MB efficientnet_v2-b0_no_quant.tflite 

In [10]:
%%bash

CUDA_VISIBLE_DEVICES='1,2,3,4' python ./eval_script.py -m ./efficientnet_v2-b0_no_quant.tflite --central_crop 0.875 -d cifar10 

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2022-04-06 02:35:42.969394: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-06 02:35:44.339947: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46904 MB memory:  -> device: 0, name: Quadro RTX 8000, pci bus id: 0000:1a:00.0, compute capability: 7.5
2022-04-06 02:35:44.341959: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46904 MB memory:  -> device: 1, name: Quadro RTX 8000, pci bus id: 0000:67:00.0, compute capability: 7.5
2022-04-06 02:35:44.343873: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525]

>>>> Using input_shape [224, 224] for TFLite model.
>>>> rescale_mode: torch
>i>>> Dataset args: {'data_name': 'cifar10', 'batch_size': 64, 'buffer_size': 1000, 'info_only': False, 'rescale_mode': 'torch', 'random_crop_min': 1.0, 'random_erasing_prob': 0.0, 'magnitude': 0, 'num_layers': 2, 'augment_kwargs': {}, 'cutmix_alpha': 0, 'eval_central_crop': 0.875, 'input_shape': [224, 224], 'mixup_alpha': 0, 'resize_antialias': True, 'resize_method': 'bicubic'}


Evaluating: 100%|██████████| 157/157 [03:35<00:00,  1.37s/it]


>>>> Accuracy top1: 0.9523 top5: 0.9987


### Need to use representative dataset for uint quantization
* Make sure that try with only conv2d to split conv2d operation done
* Performed both the operations here conv2d to split conv2d and other gelu one which probably messed up the model acc

In [18]:
from keras_cv_attention_models.imagenet import data

def representative_dataset2():
  for data in tf.data.Dataset.from_tensor_slices((images)).batch(1).take(100):
    yield [tf.dtypes.cast(data, tf.float32)]
    
def representative_dataset():
    for _ in range(100):
      data = np.random.rand(1, 224, 224, 3)
      yield [data.astype(np.float32)]
        
def representative_dataset_gen():
    train_dataset, _, _, _, _ = data.init_dataset(data_name='cifar10', input_shape=mm.input_shape[1:], batch_size=1)
    aa = train_dataset.as_numpy_iterator()
    for _ in range(100):
        # Get sample input data as a numpy array in a method of your choosing.
        # It's `[]` wrapped image input with batch_size 1, like `[np.ones([1, 224, 224, 3])]`.
        yield [aa.next()[0]]
 
mm = model_surgery.prepare_for_tflite(model)
print(np.allclose(mm(test_inputs), model(test_inputs)))
# True

converter = tf.lite.TFLiteConverter.from_keras_model(mm)
converter.optimizations = [tf.lite.Optimize.DEFAULT] # quantizes weights
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
open(mm.name + "_int8.tflite", "wb").write(converter.convert())
# 30M regnetzd32_uint8.tflite
print(np.allclose(mm(test_inputs), eval_func.TFLiteModelInterf(mm.name + '_int8.tflite')(test_inputs), atol=1e-3))
# True

# converter = tf.lite.TFLiteConverter.from_keras_model(mm)
# converter.optimizations = [tf.lite.Optimize.DEFAULT] # quantizes weights
# converter.representative_dataset = representative_dataset 
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# converter.inference_input_type = tf.uint8  # or tf.uint8
# converter.inference_output_type = tf.int8  # or tf.uint8

# open(mm.name + "_wt8int_act8uint_v3.tflite", "wb").write(converter.convert())
# print(np.allclose(mm(test_inputs), eval_func.TFLiteModelInterf(mm.name + '_wt8int_act8uint_v3.tflite')(test_inputs), atol=1e-7))

True
INFO:tensorflow:Assets written to: /tmp/tmpvi2jzucs/assets


INFO:tensorflow:Assets written to: /tmp/tmpvi2jzucs/assets
/home/local/lambda/rishabhs/anaconda3/envs/cloned_tf2.8-cudnn8.1-cudadev-11.2-py3.8/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2022-04-06 13:52:42.035749: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-04-06 13:52:42.035780: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-04-06 13:52:42.036064: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpvi2jzucs
2022-04-06 13:52:42.126051: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-04-06 13:52:42.126080: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpvi2jzucs
20

>i>>> Dataset args: {'data_name': 'cifar10', 'batch_size': 1, 'buffer_size': 1000, 'info_only': False, 'rescale_mode': 'tf', 'random_crop_min': 1.0, 'random_erasing_prob': 0.0, 'magnitude': 0, 'num_layers': 2, 'augment_kwargs': {}, 'cutmix_alpha': 0, 'eval_central_crop': 1.0, 'input_shape': (224, 224, 3), 'mixup_alpha': 0, 'resize_antialias': False, 'resize_method': 'bilinear'}


2022-04-06 13:53:05.669887: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
fully_quantize: 0, inference_type: 6, input_inference_type: 9, output_inference_type: 9


False


In [20]:
tflite_models_dir= "/home/local/lambda/rishabhs/ML/keras_cv_attention_models/cifar10_tflite_models/"
tflite_model_quant= tflite_models_dir + "efficientnet_v2-b0_int8.tflite"

interpreter = tf.lite.Interpreter(model_path=str(tflite_model_quant))
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.int8'>
output:  <class 'numpy.int8'>


### Accuracy top1: 0.1 top5: 0.4999 8INT
This is with signed INT. UINT is much better
### Accuracy top1: 0.8697 top5: 0.992 8UINT

In [21]:
%%bash

CUDA_VISIBLE_DEVICES='1,2,3,4' python ./eval_script.py -m ./efficientnet_v2-b0_int8.tflite --central_crop 0.875 -d cifar10 

2022-04-06 13:53:45.671800: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-06 13:53:47.039055: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46904 MB memory:  -> device: 0, name: Quadro RTX 8000, pci bus id: 0000:1a:00.0, compute capability: 7.5
2022-04-06 13:53:47.041280: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46904 MB memory:  -> device: 1, name: Quadro RTX 8000, pci bus id: 0000:67:00.0, compute capability: 7.5
2022-04-06 13:53:47.043287: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:G

>>>> Using input_shape [224, 224] for TFLite model.
>>>> rescale_mode: torch
>i>>> Dataset args: {'data_name': 'cifar10', 'batch_size': 64, 'buffer_size': 1000, 'info_only': False, 'rescale_mode': 'torch', 'random_crop_min': 1.0, 'random_erasing_prob': 0.0, 'magnitude': 0, 'num_layers': 2, 'augment_kwargs': {}, 'cutmix_alpha': 0, 'eval_central_crop': 0.875, 'input_shape': [224, 224], 'mixup_alpha': 0, 'resize_antialias': True, 'resize_method': 'bicubic'}


Evaluating: 100%|██████████| 157/157 [4:27:59<00:00, 102.42s/it]


>>>> Accuracy top1: 0.1 top5: 0.4999


### Conv2d to Split Conv2d
* Once the model is converted the new model outputs for a set of inputs is compared with the original model outputs. (bb)

### Model converted to a TFLite model (8uint quantization done) weights are 8int
* Again the converted the model outputs for a set of inputs is compared with the Split Conv2d model.

In [22]:
# Here I am only using any conv2d to split conv2d surgery before converting the model into quantized one
bb = model_surgery.convert_groups_conv2d_2_split_conv2d(model)  # converts all `Conv2D` using `groups` to `SplitConv2D`
#mm = model
test_inputs = np.random.uniform(size=[1, *bb.input_shape[1:]])
print(np.allclose(bb(test_inputs), model(test_inputs)))
# True

converter = tf.lite.TFLiteConverter.from_keras_model(bb)
converter.optimizations = [tf.lite.Optimize.DEFAULT] # quantizes weights
converter.representative_dataset = representative_dataset_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
open(bb.name + "_nosurg_uint8.tflite", "wb").write(converter.convert())
# 30M regnetzd32_uint8.tflite
print(np.allclose(bb(test_inputs), eval_func.TFLiteModelInterf(bb.name + '_nosurg_uint8.tflite')(test_inputs), atol=1e-3))

True
INFO:tensorflow:Assets written to: /tmp/tmphr6c2kbj/assets


INFO:tensorflow:Assets written to: /tmp/tmphr6c2kbj/assets
/home/local/lambda/rishabhs/anaconda3/envs/cloned_tf2.8-cudnn8.1-cudadev-11.2-py3.8/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2022-04-06 20:47:59.982419: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-04-06 20:47:59.982450: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-04-06 20:47:59.982732: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmphr6c2kbj
2022-04-06 20:48:00.070743: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-04-06 20:48:00.070772: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmphr6c2kbj
20

>i>>> Dataset args: {'data_name': 'cifar10', 'batch_size': 1, 'buffer_size': 1000, 'info_only': False, 'rescale_mode': 'tf', 'random_crop_min': 1.0, 'random_erasing_prob': 0.0, 'magnitude': 0, 'num_layers': 2, 'augment_kwargs': {}, 'cutmix_alpha': 0, 'eval_central_crop': 1.0, 'input_shape': (224, 224, 3), 'mixup_alpha': 0, 'resize_antialias': False, 'resize_method': 'bilinear'}


2022-04-06 20:48:24.736398: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
fully_quantize: 0, inference_type: 6, input_inference_type: 3, output_inference_type: 3


False


In [23]:
tflite_models_dir= "/home/local/lambda/rishabhs/ML/keras_cv_attention_models/cifar10_tflite_models/"
tflite_model_quant= tflite_models_dir + "efficientnet_v2-b0_nosurg_uint8.tflite"

interpreter = tf.lite.Interpreter(model_path=str(tflite_model_quant))
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>


In [24]:
%%bash

CUDA_VISIBLE_DEVICES='1,2,3,4' python ./eval_script.py -m ./efficientnet_v2-b0_nosurg_uint8.tflite --central_crop 0.875 -d cifar10 

2022-04-06 20:49:57.499590: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-06 20:49:58.871090: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46904 MB memory:  -> device: 0, name: Quadro RTX 8000, pci bus id: 0000:1a:00.0, compute capability: 7.5
2022-04-06 20:49:58.873115: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46904 MB memory:  -> device: 1, name: Quadro RTX 8000, pci bus id: 0000:67:00.0, compute capability: 7.5
2022-04-06 20:49:58.874986: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:G

>>>> Using input_shape [224, 224] for TFLite model.
>>>> rescale_mode: torch
>i>>> Dataset args: {'data_name': 'cifar10', 'batch_size': 64, 'buffer_size': 1000, 'info_only': False, 'rescale_mode': 'torch', 'random_crop_min': 1.0, 'random_erasing_prob': 0.0, 'magnitude': 0, 'num_layers': 2, 'augment_kwargs': {}, 'cutmix_alpha': 0, 'eval_central_crop': 0.875, 'input_shape': [224, 224], 'mixup_alpha': 0, 'resize_antialias': True, 'resize_method': 'bicubic'}


Evaluating: 100%|██████████| 157/157 [4:25:04<00:00, 101.30s/it]


>>>> Accuracy top1: 0.8693 top5: 0.991


## JUNK CODE BELOW

In [ ]:
import tensorflow as tf
print(tf.__version__)
#from keras_cv_attention_models.efficientnet import progressive_train_test
#os.environ['CUDA_VISIBLE_DEVICES'] = '3'
os.environ['TF_XLA_FLAGS'] = "--tf_xla_auto_jit=2"

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
!python progressive_train_script.py \
-m efficientnet.EfficientNetV2B0 --summary --pretrained imagenet -d cifar10 --lr_decay_steps 36 -s effv2b0_preprocTrue_cifar10_224_progressive \
--progressive_epochs 10 20 30 -1 \
--progressive_input_shapes 128 160 192 224 \
--progressive_dropouts 0.1 0.2 0.3 0.4 \
--progressive_magnitudes 5 8 12 15 \
--progressive_batch_sizes 240 \
--seed 0

In [ ]:
%%bash

CUDA_VISIBLE_DEVICES='3' TF_XLA_FLAGS="--tf_xla_auto_jit=2" python ./progressive_train_script.py \
-m efficientnet.EfficientNetV2B0 --pretrained imagenet -d cifar10 --lr_decay_steps 36 -s effv2b0_cifar10_224_progressive \
--progressive_epochs 10 20 30 -1 \
--progressive_input_shapes 128 160 192 224 \
--progressive_dropouts 0.1 0.2 0.3 0.4 \
--progressive_magnitudes 5 8 12 15 \
--progressive_batch_sizes 240 \
--seed 0